# # Intelligent Academic Advisor System
# ## Optimization-Based Course Recommendation with ML Risk Prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings

# Suppress sklearn warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

# Add src to path
sys.path.append('.')

# Import our modules
from src.data_loader import DataLoader
from src.risk_predictor import CourseFailurePredictor
from src.optimizer import CourseOptimizer
from src.multi_semester_planner import MultiSemesterPlanner
from src.evaluator import AdvisorEvaluator
from src.explanation_generator import ExplanationGenerator

print("✅ All libraries imported successfully!")


In [ ]:
print("📂 Loading academic data...\n")

loader = DataLoader("data")
loader.load_all()

courses = loader.courses
prereqs = loader.prereqs
students = loader.students
student_courses = loader.student_courses
rules = loader.rules
G = loader.prereq_graph
rules_dict = loader.get_rules_dict()

print(f"\n📊 Data Summary:")
print(f"   • Total Courses: {len(courses)}")
print(f"   • Total Students: {len(students)}")


In [ ]:
print("🔗 Visualizing prerequisite dependencies...\n")

plt.figure(figsize=(16, 12))
import networkx as nx

pos = nx.spring_layout(G, k=1.0, iterations=50, seed=42)
nx.draw(G, pos, 
        with_labels=True,
        node_color='lightblue',
        node_size=2000,
        font_size=8,
        font_weight='bold',
        arrowsize=12,
        edge_color='gray',
        alpha=0.8)

plt.title("Air University BS CS Prerequisite Dependency Graph", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
print("🎓 Training Machine Learning Risk Predictor...\n")

risk_model = CourseFailurePredictor()

# Check if model already exists
model_path = Path("models/risk_predictor.pkl")
if model_path.exists():
    print("📂 Loading existing model...")
    risk_model.load(model_path)
else:
    print("🔨 Training new model...")
    risk_model.train(students, student_courses, courses, G)
    risk_model.save(model_path)

print("\n✅ Risk prediction model ready!")


In [ ]:
print("⚙️ Initializing system components...\n")

optimizer = CourseOptimizer(rules_dict)
planner = MultiSemesterPlanner(courses, G, rules_dict)
evaluator = AdvisorEvaluator(courses, rules_dict)
explainer = ExplanationGenerator()

print("✅ All components initialized!\n")


In [ ]:
print("👥 Available Students:\n")
for i, row in students.iterrows():
    print(f"{i+1}. {row['student_id']} - CGPA: {row['cgpa']:.2f}, Semester: {row['current_semester']}")

# SELECT A STUDENT HERE (change this to test different students)
selected_student_id = "CS2025-001"  # ⬅️ CHANGE THIS TO TEST OTHERS

print(f"\n✨ Selected: {selected_student_id}\n")

# GET STUDENT PROFILE (this creates the student_profile variable)
student_profile = loader.get_student_profile(selected_student_id)

print(f"📋 Student Profile:")
print(f"   • Student ID: {student_profile['student_id']}")
print(f"   • CGPA: {student_profile['student']['cgpa']:.2f}")
print(f"   • Current Semester: {student_profile['student']['current_semester']}")
print(f"   • Completed Courses: {len(student_profile['completed_courses'])}")
print(f"   • Backlogs: {len(student_profile['backlogs'])}")
if student_profile['backlogs']:
    print(f"      → {sorted(student_profile['backlogs'])}")

print("\n✅ Student profile loaded - ready for recommendation!")

In [ ]:
next_semester = student_profile['student']['current_semester'] + 1

eligible_df = loader.get_eligible_courses(
    student_profile['completed_courses'],
    next_semester,
    student_profile['backlogs']
)

print(f"\n📚 Eligible Courses for Semester {next_semester}: {len(eligible_df)}")
if not eligible_df.empty:
    display(eligible_df[['course_code', 'course_name', 'credits', 'difficulty', 'semester']].head(10))
else:
    print("⚠️ No eligible courses found!")

In [ ]:
print("\n🔮 Predicting failure risk for eligible courses...\n")

risk_scores = risk_model.predict_batch(
    eligible_df,
    student_profile,
    G,
    next_semester
)

# Add risk to eligible courses
eligible_df_with_risk = eligible_df.copy()
eligible_df_with_risk['risk_score'] = eligible_df_with_risk['course_code'].map(risk_scores)

# Show highest risk courses
print("⚠️ Highest Risk Courses:")
high_risk = eligible_df_with_risk.nlargest(5, 'risk_score')[
    ['course_code', 'course_name', 'difficulty', 'risk_score']
]
display(high_risk)

In [ ]:
next_semester = student_profile['student']['current_semester'] + 1

eligible_df = loader.get_eligible_courses(
    student_profile['completed_courses'],
    next_semester,
    student_profile['backlogs']
)

print(f"\n📚 Eligible Courses for Semester {next_semester}: {len(eligible_df)}")
if not eligible_df.empty:
    display(eligible_df[['course_code', 'course_name', 'credits', 'difficulty', 'semester']].head(10))
else:
    print("⚠️ No eligible courses found!")

In [ ]:
print("\n🎯 Generating Optimal Course Recommendation...\n")

recommended_df, metadata = optimizer.recommend(
    eligible_df,
    student_profile,
    risk_scores=risk_scores
)

if recommended_df.empty:
    print("❌ No recommendation could be generated!")
    print(f"Status: {metadata.get('status', 'unknown')}")
else:
    print("✅ Recommendation generated successfully!\n")
    
    # Generate full report
    report = explainer.generate_full_report(
        recommended_df,
        student_profile,
        metadata
    )
    
    print(report)

In [ ]:
# ============================================================================
# INTERACTIVE WIDGETS
# ============================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from tabulate import tabulate

# ============================================================================
# WIDGET DEFINITIONS
# ============================================================================

# Title
title_widget = widgets.HTML(
    value="""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
        <h2 style='color: white; margin: 0; text-align: center;'>
            🎓 Intelligent Academic Advisor System
        </h2>
        <p style='color: white; margin: 5px 0 0 0; text-align: center; opacity: 0.9;'>
            AI-Powered Course Recommendation with ML Risk Prediction
        </p>
    </div>
    """,
    layout=widgets.Layout(width='100%')
)

# Student Selection
student_dropdown = widgets.Dropdown(
    options=[(f"{row['student_id']} (CGPA: {row['cgpa']:.2f}, Sem: {row['current_semester']})", 
              row['student_id'])
             for _, row in students.iterrows()],
    description='Select Student:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px')
)

# Weight Control (Optional - Advanced Mode)
use_custom_weights = widgets.Checkbox(
    value=False,
    description='Use Custom Weights (Advanced)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

weight_progress = widgets.FloatSlider(
    value=10.0, min=1, max=30, step=1,
    description='Progress Weight:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

weight_retake = widgets.FloatSlider(
    value=30.0, min=5, max=70, step=2,
    description='Retake Priority:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

weight_difficulty = widgets.FloatSlider(
    value=2.0, min=0.5, max=8, step=0.2,
    description='Difficulty Penalty:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

weight_risk = widgets.FloatSlider(
    value=5.0, min=0.5, max=15, step=0.5,
    description='Risk Penalty:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

# Enable/disable weight sliders based on checkbox
def on_custom_weights_change(change):
    enabled = change['new']
    weight_progress.disabled = not enabled
    weight_retake.disabled = not enabled
    weight_difficulty.disabled = not enabled
    weight_risk.disabled = not enabled

use_custom_weights.observe(on_custom_weights_change, names='value')

# Feature Selection
show_comparison = widgets.Checkbox(
    value=True,
    description='Show Baseline Comparison',
    style={'description_width': 'initial'}
)

show_multi_semester = widgets.Checkbox(
    value=True,
    description='Show Multi-Semester Plan',
    style={'description_width': 'initial'}
)

show_visualizations = widgets.Checkbox(
    value=True,
    description='Show Visualizations',
    style={'description_width': 'initial'}
)

# Action Button
generate_button = widgets.Button(
    description='🚀 Generate Recommendation',
    button_style='success',
    tooltip='Click to generate course recommendation',
    layout=widgets.Layout(width='300px', height='50px'),
    style={'font_weight': 'bold'}
)

# Output Area
output_area = widgets.Output(
    layout=widgets.Layout(
        border='2px solid #e1e4e8',
        border_radius='8px',
        padding='15px',
        margin='15px 0'
    )
)

# Progress indicator
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Processing:',
    bar_style='info',
    style={'bar_color': '#667eea', 'description_width': '100px'},
    layout=widgets.Layout(width='500px', visibility='hidden')
)

# ============================================================================
# MAIN RECOMMENDATION FUNCTION
# ============================================================================

def generate_recommendation(button):
    """Main function to generate and display recommendation"""
    
    with output_area:
        clear_output(wait=True)
        
        # Show progress
        progress_bar.layout.visibility = 'visible'
        progress_bar.value = 0
        
        try:
            # Get selected student
            student_id = student_dropdown.value
            progress_bar.value = 10
            
            print("="*70)
            print(f"🎯 GENERATING RECOMMENDATION FOR {student_id}")
            print("="*70 + "\n")
            
            # Get student profile
            student_profile = loader.get_student_profile(student_id)
            student = student_profile['student']
            
            # Display student info
            print(f"📋 Student Profile:")
            print(f"   • CGPA: {student['cgpa']:.2f}")
            print(f"   • Current Semester: {student['current_semester']}")
            print(f"   • Completed Courses: {len(student_profile['completed_courses'])}")
            print(f"   • Backlogs: {len(student_profile['backlogs'])}")
            if student_profile['backlogs']:
                print(f"     → {', '.join(sorted(student_profile['backlogs']))}")
            print()
            
            progress_bar.value = 20
            
            # Get eligible courses
            next_semester = student['current_semester'] + 1
            eligible_df = loader.get_eligible_courses(
                student_profile['completed_courses'],
                next_semester,
                student_profile['backlogs']
            )
            
            if eligible_df.empty:
                print("❌ No eligible courses found for next semester!")
                progress_bar.layout.visibility = 'hidden'
                return
            
            print(f"✅ Found {len(eligible_df)} eligible courses for Semester {next_semester}\n")
            progress_bar.value = 40
            
            # Predict risks
            print("🔮 Predicting course failure risks using ML model...")
            risk_scores = risk_model.predict_batch(
                eligible_df, student_profile, G, next_semester
            )
            print(f"✅ Risk prediction complete\n")
            progress_bar.value = 60
            
            # Get weights
            if use_custom_weights.value:
                weights = {
                    'progress': weight_progress.value,
                    'retake': weight_retake.value,
                    'difficulty': weight_difficulty.value,
                    'risk': weight_risk.value
                }
                print("⚙️ Using custom weights")
            else:
                weights = None
                print("⚙️ Using adaptive weights (auto-calculated)")
            
            # Generate recommendation
            print("🎯 Optimizing course selection...\n")
            recommended_df, metadata = optimizer.recommend(
                eligible_df, student_profile, risk_scores, weights
            )
            
            if recommended_df.empty:
                print(f"❌ Could not generate recommendation: {metadata.get('status', 'unknown')}")
                progress_bar.layout.visibility = 'hidden'
                return
            
            progress_bar.value = 80
            
            # ================================================================
            # DISPLAY RESULTS
            # ================================================================
            
            print("="*70)
            print("📚 RECOMMENDED COURSES")
            print("="*70 + "\n")
            
            # Generate explanations
            explanation_df = explainer.generate_course_explanations(
                recommended_df, student_profile, metadata
            )
            
            print(tabulate(explanation_df, headers='keys', tablefmt='grid', showindex=False))
            
            # Summary
            print(f"\n📊 SUMMARY:")
            print(f"   • Total Credits: {metadata['total_credits']}/{metadata['max_credits']}")
            print(f"   • Number of Courses: {metadata['num_courses']}")
            print(f"   • Backlogs Cleared: {metadata['backlogs_cleared']}")
            print(f"   • Average Difficulty: {metadata['avg_difficulty']:.1f}/10")
            print(f"   • Average Risk: {metadata['avg_risk']:.1%}")
            
            if 'weights_used' in metadata:
                print(f"\n⚙️ WEIGHTS USED:")
                for key, value in metadata['weights_used'].items():
                    print(f"   • {key.capitalize()}: {value:.1f}")
            
            # ================================================================
            # BASELINE COMPARISON
            # ================================================================
            
            if show_comparison.value:
                print("\n" + "="*70)
                print("📊 BASELINE COMPARISON")
                print("="*70 + "\n")
                
                comparison_df = evaluator.compare_methods(
                    eligible_df,
                    recommended_df['course_code'].tolist(),
                    student_profile,
                    metadata['max_credits'],
                    risk_scores
                )
                
                print(tabulate(
                    comparison_df[['total_credits', 'backlogs_cleared', 'avg_risk', 'quality_score']],
                    headers='keys',
                    tablefmt='grid',
                    floatfmt='.2f'
                ))
                
                best = comparison_df.index[0]
                if best == 'Our System':
                    print("\n✅ Our system provides the BEST recommendation!")
                else:
                    print(f"\n⚠️ Our system ranked #{comparison_df.loc['Our System', 'rank']}")
            
            # ================================================================
            # MULTI-SEMESTER PLAN
            # ================================================================
            
            if show_multi_semester.value:
                print("\n" + "="*70)
                print("🗓️ MULTI-SEMESTER GRADUATION PLAN")
                print("="*70 + "\n")
                
                future_plan = planner.plan_graduation_path(
                    student_profile,
                    student_profile['completed_courses'],
                    num_semesters=4,
                    risk_predictor=risk_model
                )
                
                grad_sem = planner.estimate_graduation_semester(
                    student_profile,
                    student_profile['completed_courses']
                )
                
                print(f"🎓 Estimated Graduation: Semester {grad_sem}\n")
                
                for plan in future_plan:
                    print(f"📅 Semester {plan['semester']} - {plan['total_credits']} credits")
                    print("-" * 60)
                    if plan['courses']:
                        for code in plan['courses'][:6]:
                            course_name = courses[courses['course_code'] == code]['course_name'].values[0]
                            print(f"   • {code}: {course_name}")
                        if len(plan['courses']) > 6:
                            print(f"   ... and {len(plan['courses']) - 6} more courses")
                    else:
                        print(f"   {plan.get('note', 'No courses planned')}")
                    print()
                
                # Progress
                progress_info = planner.calculate_progress_percentage(
                    student_profile['completed_courses']
                )
                print(f"📊 Degree Progress:")
                print(f"   • Completed: {progress_info['total_credits_completed']}/{progress_info['total_credits_required']} credits")
                print(f"   • Percentage: {progress_info['percentage_complete']:.1f}%")
            
            # ================================================================
            # VISUALIZATIONS
            # ================================================================
            
            if show_visualizations.value:
                print("\n" + "="*70)
                print("📈 VISUALIZATIONS")
                print("="*70 + "\n")
                
                fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                
                # 1. Recommended Courses by Risk
                ax1 = axes[0, 0]
                colors = ['#2ecc71' if r < 0.3 else '#f39c12' if r < 0.5 else '#e74c3c' 
                         for r in recommended_df['risk_score']]
                ax1.barh(recommended_df['course_code'], recommended_df['risk_score'], color=colors)
                ax1.set_xlabel('Risk Score')
                ax1.set_title('Risk Assessment by Course')
                ax1.axvline(x=0.3, color='green', linestyle='--', alpha=0.5, label='Low Risk')
                ax1.axvline(x=0.5, color='orange', linestyle='--', alpha=0.5, label='Moderate')
                ax1.legend()
                
                # 2. Credits vs Difficulty
                ax2 = axes[0, 1]
                scatter = ax2.scatter(recommended_df['difficulty'], recommended_df['credits'],
                                     c=recommended_df['risk_score'], cmap='RdYlGn_r',
                                     s=200, alpha=0.6, edgecolors='black')
                ax2.set_xlabel('Difficulty')
                ax2.set_ylabel('Credits')
                ax2.set_title('Workload Distribution')
                plt.colorbar(scatter, ax=ax2, label='Risk Score')
                
                for _, row in recommended_df.iterrows():
                    ax2.annotate(row['course_code'], 
                               (row['difficulty'], row['credits']),
                               fontsize=8, ha='center')
                
                # 3. Baseline Comparison (if enabled)
                if show_comparison.value:
                    ax3 = axes[1, 0]
                    methods = comparison_df.index.tolist()
                    scores = comparison_df['quality_score'].tolist()
                    colors_bar = ['#2ecc71' if m == 'Our System' else '#95a5a6' for m in methods]
                    ax3.bar(methods, scores, color=colors_bar)
                    ax3.set_ylabel('Quality Score')
                    ax3.set_title('Quality Comparison')
                    ax3.tick_params(axis='x', rotation=45)
                else:
                    axes[1, 0].axis('off')
                
                # 4. Degree Progress (if multi-semester enabled)
                if show_multi_semester.value:
                    ax4 = axes[1, 1]
                    progress_info = planner.calculate_progress_percentage(
                        student_profile['completed_courses']
                    )
                    completed_pct = progress_info['percentage_complete']
                    remaining_pct = 100 - completed_pct
                    
                    ax4.pie([completed_pct, remaining_pct],
                           labels=['Completed', 'Remaining'],
                           autopct='%1.1f%%',
                           colors=['#3498db', '#ecf0f1'],
                           startangle=90)
                    ax4.set_title(f'Degree Completion ({progress_info["total_credits_completed"]}/{progress_info["total_credits_required"]} credits)')
                else:
                    axes[1, 1].axis('off')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            
            print("\n" + "="*70)
            print("✅ RECOMMENDATION COMPLETE!")
            print("="*70)
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
        
        finally:
            # Hide progress bar
            progress_bar.layout.visibility = 'hidden'

# Attach function to button
generate_button.on_click(generate_recommendation)

# ============================================================================
# LAYOUT & DISPLAY
# ============================================================================

# Create collapsible advanced settings
advanced_settings = widgets.Accordion(children=[
    widgets.VBox([
        use_custom_weights,
        weight_progress,
        weight_retake,
        weight_difficulty,
        weight_risk
    ])
])
advanced_settings.set_title(0, '⚙️ Advanced: Custom Weights')
advanced_settings.selected_index = None  # Collapsed by default

# Feature checkboxes
feature_box = widgets.HBox([
    show_comparison,
    show_multi_semester,
    show_visualizations
], layout=widgets.Layout(margin='15px 0'))

# Main layout
main_interface = widgets.VBox([
    title_widget,
    widgets.HTML("<hr style='border: 1px solid #e1e4e8; margin: 10px 0;'>"),
    widgets.HTML("<h3 style='margin: 15px 0 10px 0;'>Student Selection</h3>"),
    student_dropdown,
    widgets.HTML("<h3 style='margin: 20px 0 10px 0;'>Options</h3>"),
    feature_box,
    advanced_settings,
    widgets.HTML("<div style='margin: 20px 0;'></div>"),
    generate_button,
    progress_bar,
    output_area
], layout=widgets.Layout(
    padding='20px',
    border='3px solid #667eea',
    border_radius='15px',
    background_color='#f8f9fa'
))

# ============================================================================
# DISPLAY THE INTERFACE
# ============================================================================

print("="*70)
print("🎉 INTERACTIVE ADVISOR INTERFACE LOADED!")
print("="*70)
print("\n👇 Use the interface below to generate recommendations:\n")

display(main_interface)

In [ ]:
next_semester = student_profile['student']['current_semester'] + 1

eligible_df = loader.get_eligible_courses(
    student_profile['completed_courses'],
    next_semester,
    student_profile['backlogs']
)

print(f"\n📚 Eligible Courses for Semester {next_semester}: {len(eligible_df)}")
if not eligible_df.empty:
    display(eligible_df[['course_code', 'course_name', 'credits', 'difficulty', 'semester']].head(10))


In [ ]:
print("\n🔮 Predicting failure risk for eligible courses...\n")

risk_scores = risk_model.predict_batch(
    eligible_df,
    student_profile,
    G,
    next_semester
)

# Add risk to eligible courses
eligible_df_with_risk = eligible_df.copy()
eligible_df_with_risk['risk_score'] = eligible_df_with_risk['course_code'].map(risk_scores)

# Show highest risk courses
print("⚠️ Highest Risk Courses:")
high_risk = eligible_df_with_risk.nlargest(5, 'risk_score')[
    ['course_code', 'course_name', 'difficulty', 'risk_score']
]
display(high_risk)


In [ ]:
print("\n🎯 Generating Optimal Course Recommendation...\n")

recommended_df, metadata = optimizer.recommend(
    eligible_df,
    student_profile,
    risk_scores=risk_scores
)

if recommended_df.empty:
    print("❌ No recommendation could be generated!")
    print(f"Status: {metadata.get('status', 'unknown')}")
else:
    print("✅ Recommendation generated successfully!\n")
    
    # Generate full report
    report = explainer.generate_full_report(
        recommended_df,
        student_profile,
        metadata
    )
    
    print(report)


In [ ]:
if not recommended_df.empty:
    print("\n" + "="*70)
    print("📊 COMPARING WITH BASELINE METHODS")
    print("="*70 + "\n")
    
    comparison_df = evaluator.compare_methods(
        eligible_df,
        recommended_df['course_code'].tolist(),
        student_profile,
        metadata['max_credits'],
        risk_scores
    )
    
    evaluator.print_comparison_report(comparison_df)
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Quality score comparison
    comparison_df['quality_score'].plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title('Quality Score')
    axes[0].set_ylabel('Score')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Credits comparison
    comparison_df['total_credits'].plot(kind='bar', ax=axes[1], color='coral')
    axes[1].set_title('Total Credits')
    axes[1].set_ylabel('Credits')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Risk comparison
    comparison_df['avg_risk'].plot(kind='bar', ax=axes[2], color='salmon')
    axes[2].set_title('Average Risk')
    axes[2].set_ylabel('Risk Score')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()


In [ ]:
if not recommended_df.empty:
    print("\n" + "="*70)
    print("🗓️ MULTI-SEMESTER GRADUATION PATH")
    print("="*70 + "\n")
    
    # Generate 4-semester plan
    future_plan = planner.plan_graduation_path(
        student_profile,
        student_profile['completed_courses'],
        num_semesters=4,
        risk_predictor=risk_model
    )
    
    # Estimate graduation
    grad_semester = planner.estimate_graduation_semester(
        student_profile,
        student_profile['completed_courses']
    )
    
    print(f"🎓 Estimated Graduation: Semester {grad_semester}\n")
    
    # Display plan
    for i, plan in enumerate(future_plan, 1):
        print(f"\n📅 Semester {plan['semester']} ({plan['total_credits']} credits):")
        if plan['courses']:
            for code in plan['courses'][:5]:  # Show first 5
                course_name = courses[courses['course_code'] == code]['course_name'].values[0]
                print(f"   • {code}: {course_name}")
            if len(plan['courses']) > 5:
                print(f"   ... and {len(plan['courses']) - 5} more courses")
        else:
            print(f"   {plan.get('note', 'No courses planned')}")
    
    # Progress visualization
    progress = planner.calculate_progress_percentage(student_profile['completed_courses'])
    
    print(f"\n📊 Degree Progress:")
    print(f"   • Credits Completed: {progress['total_credits_completed']}/{progress['total_credits_required']}")
    print(f"   • Percentage Complete: {progress['percentage_complete']:.1f}%")
    print(f"   • Credits Remaining: {progress['credits_remaining']}")

In [ ]:
if not recommended_df.empty:
    print("\n" + "="*70)
    print("🔀 ALTERNATIVE RECOMMENDATIONS")
    print("="*70 + "\n")
    
    alternatives = optimizer.generate_alternatives(
        eligible_df,
        student_profile,
        risk_scores,
        num_alternatives=3
    )
    
    for i, (alt_rec, alt_meta) in enumerate(alternatives, 1):
        print(f"\n{'='*60}")
        print(f"Option {i}: {alt_meta['profile']}")
        print(f"{'='*60}")
        print(f"Credits: {alt_meta['total_credits']}, Risk: {alt_meta['avg_risk']:.1%}")
        print(f"Courses:")
        for _, course in alt_rec.iterrows():
            print(f"   • {course['course_code']}: {course['course_name']}")


In [ ]:
print("\n" + "="*70)
print("📈 SYSTEM PERFORMANCE ACROSS ALL STUDENTS")
print("="*70 + "\n")

def get_recommendation_for_student(student_profile):
    next_sem = student_profile['student']['current_semester'] + 1

    eligible = loader.get_eligible_courses(
        student_profile['completed_courses'],
        next_sem,
        student_profile['backlogs']
    )

    if eligible.empty:
        return pd.DataFrame(), {
            'status': 'no_eligible',
            'total_credits': 0,
            'avg_risk': 0.0,
            'history': []
        }

    risks = risk_model.predict_batch(eligible, student_profile, G, next_sem)

    rec_df, metrics = optimizer.recommend(
        eligible,
        student_profile,
        risks
    )

    # 🔴 GUARANTEE keys exist
    metrics.setdefault('history', [])
    metrics.setdefault('avg_risk', 0.0)
    metrics.setdefault('total_credits', 0)

    return rec_df, metrics

# Run batch evaluation
batch_results = evaluator.batch_evaluate(
    students,
    get_recommendation_for_student,
    student_courses,
    courses,
    G
)

print("Batch Evaluation Results:\n")
display(batch_results)

# Summary statistics
print(f"\n📊 Summary Statistics:")
successful = batch_results[batch_results['status'] == 'success']

if not successful.empty:
    print(f"   • Average Credits Recommended: {successful['recommended_credits'].mean():.1f}")
    print(f"   • Average Risk: {successful['avg_risk'].mean():.2%}")
else:
    print("   • No successful recommendations to analyze")
print(f"   • Average Risk: {batch_results['avg_risk'].mean():.2%}")
print(f"   • Students with Backlogs: {(batch_results['backlogs_count'] > 0).sum()}")

# Plot CGPA vs recommended credits
plt.figure(figsize=(10, 6))
plt.scatter(batch_results['cgpa'], batch_results['recommended_credits'], 
           s=100, alpha=0.6, c=batch_results['backlogs_count'], cmap='Reds')
plt.xlabel('Student CGPA')
plt.ylabel('Recommended Credits')
plt.title('Recommended Credits vs CGPA (color = number of backlogs)')
plt.colorbar(label='Backlogs')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
if not recommended_df.empty:
    print("\n💾 Exporting recommendation to CSV...")
    
    # Create output directory
    Path("output").mkdir(exist_ok=True)
    
    # Export recommendation
    output_file = f"output/recommendation_{selected_student_id}_sem{next_semester}.csv"
    recommended_df.to_csv(output_file, index=False)
    
    print(f"✅ Saved to: {output_file}")
    
    # Also save the full report as text
    report_file = f"output/report_{selected_student_id}_sem{next_semester}.txt"
    with open(report_file, 'w') as f:
        f.write(explainer.generate_full_report(
            recommended_df, student_profile, metadata, comparison_df
        ))
    
    print(f"✅ Report saved to: {report_file}")




# ## Summary
# 
# This notebook demonstrates:
# 1. ✅ ML-based risk prediction
# 2. ✅ Optimization-based course selection
# 3. ✅ Multi-semester graduation planning
# 4. ✅ Baseline comparison & evaluation
# 5. ✅ Explainable recommendations
# 
# **Next Steps:**
# - Run CLI: `python advisor_cli.py`
# - Adjust student selection in Cell 6
# - Experiment with different weight profiles
# - Add more students to CSV files


In [ ]:
print("\n" + "="*70)
print("✅ NOTEBOOK COMPLETE!")
print("="*70)

In [ ]:
# ============================================================================
# FIXED NOTEBOOK CELLS - RUN IN THIS EXACT ORDER
# ============================================================================

# %% CELL 1: Import Libraries

# %% CELL 2: Load Data

# %% CELL 3: Train Risk Prediction Model

# %% CELL 4: Initialize System Components

# %% CELL 5: Select Student (IMPORTANT - DON'T SKIP!)


# %% CELL 6: Get Eligible Courses (NOW student_profile exists!)


# %% CELL 7: Predict Risk Scores


# %% CELL 8: Generate Recommendation
print("\n🎯 Generating Optimal Course Recommendation...\n")

recommended_df, metadata = optimizer.recommend(
    eligible_df,
    student_profile,
    risk_scores=risk_scores
)

if recommended_df.empty:
    print("❌ No recommendation could be generated!")
    print(f"Status: {metadata.get('status', 'unknown')}")
else:
    print("✅ Recommendation generated successfully!\n")
    
    # Generate full report
    report = explainer.generate_full_report(
        recommended_df,
        student_profile,
        metadata
    )
    
    print(report)

# %% CELL 9: Compare with Baselines
if not recommended_df.empty:
    print("\n" + "="*70)
    print("📊 COMPARING WITH BASELINE METHODS")
    print("="*70 + "\n")
    
    comparison_df = evaluator.compare_methods(
        eligible_df,
        recommended_df['course_code'].tolist(),
        student_profile,
        metadata['max_credits'],
        risk_scores
    )
    
    evaluator.print_comparison_report(comparison_df)

# %% CELL 10: Multi-Semester Planning
if not recommended_df.empty:
    print("\n" + "="*70)
    print("🗓️ MULTI-SEMESTER GRADUATION PATH")
    print("="*70 + "\n")
    
    # Generate 4-semester plan
    future_plan = planner.plan_graduation_path(
        student_profile,
        student_profile['completed_courses'],
        num_semesters=4,
        risk_predictor=risk_model
    )
    
    # Estimate graduation
    grad_semester = planner.estimate_graduation_semester(
        student_profile,
        student_profile['completed_courses']
    )
    
    print(f"🎓 Estimated Graduation: Semester {grad_semester}\n")
    
    # Display plan
    for i, plan in enumerate(future_plan, 1):
        print(f"\n📅 Semester {plan['semester']} ({plan['total_credits']} credits):")
        if plan['courses']:
            for code in plan['courses'][:5]:
                course_name = courses[courses['course_code'] == code]['course_name'].values[0]
                print(f"   • {code}: {course_name}")
            if len(plan['courses']) > 5:
                print(f"   ... and {len(plan['courses']) - 5} more courses")
        else:
            print(f"   {plan.get('note', 'No courses planned')}")

# %% CELL 11: INTERACTIVE WIDGETS (OPTIONAL - BEST INTERFACE!)
# Copy the widget code from the artifact I created earlier
# This provides a beautiful UI for testing different students